In [18]:
import os 
import cv2 
import numpy as np 
import tensorflow as tf 
import sys

gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.98)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

  
# # Import utilites 
# from object_detection.utils import label_map_util 
# from object_detection.utils import visualization_utils as vis_util

In [ ]:
# # Name of the directory containing the object detection module we're using 
# #MODEL_NAME = 'ssd_mobilenet_v1_coco_11_06_2017' # The path to the directory where frozen_inference_graph is stored. 
# IMAGE_NAME = 'amb1.jpg'  # The path to the image in which the object has to be detected. !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
  
# # Grab path to current working directory 
# CWD_PATH = os.getcwd() 
  
# # Path to frozen detection graph .pb file, which contains the model that is used 
# # for object detection. 
# PATH_TO_CKPT = 'frozen_inference_graph.pb'
# #PATH_TO_CKPT = 'frozen_inference_graph.pb'
  
# # Path to label map file 
# PATH_TO_LABELS = 'annotations/label_map.pbtxt'
  
# # Path to image 
# PATH_TO_IMAGE =  'amb1.jpg'#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
  
# # Number of classes the object detector can identify 
# NUM_CLASSES = 1
  
# # Load the label map. 
# # Label maps map indices to category names, so that when our convolution 
# # network predicts `5`, we know that this corresponds to `king`. 
# # Here we use internal utility functions, but anything that returns a 
# # dictionary mapping integers to appropriate string labels would be fine 
# label_map = label_map_util.load_labelmap(PATH_TO_LABELS) 
# categories = label_map_util.convert_label_map_to_categories( 
#         label_map, max_num_classes = NUM_CLASSES, use_display_name = True) 
# category_index = label_map_util.create_category_index(categories) 
  
# # Load the Tensorflow model into memory. 
# detection_graph = tf.Graph() 
# with detection_graph.as_default(): 
#     od_graph_def = tf.compat.v1.GraphDef()
#     with tf.io.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
#         serialized_graph = fid.read() 
#         od_graph_def.ParseFromString(serialized_graph) 
#         tf.import_graph_def(od_graph_def, name ='') 
  
#     sess = tf.compat.v1.Session(graph = detection_graph)
  
# # Define input and output tensors (i.e. data) for the object detection classifier 
  
# # Input tensor is the image 
# image_tensor = detection_graph.get_tensor_by_name('image_tensor:0') 
  
# # Output tensors are the detection boxes, scores, and classes 
# # Each box represents a part of the image where a particular object was detected 
# detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0') 
  
# # Each score represents level of confidence for each of the objects. 
# # The score is shown on the result image, together with the class label. 
# detection_scores = detection_graph.get_tensor_by_name('detection_scores:0') 
# detection_classes = detection_graph.get_tensor_by_name('detection_classes:0') 
  
# # Number of objects detected 
# num_detections = detection_graph.get_tensor_by_name('num_detections:0') 
  
# # Load image using OpenCV and 
# # expand image dimensions to have shape: [1, None, None, 3] 
# # i.e. a single-column array, where each item in the column has the pixel RGB value
# #########################################################################################################


# #########################################################################################################
# image = cv2.imread(PATH_TO_IMAGE)
# image_expanded = np.expand_dims(image, axis = 0) 
  
# # Perform the actual detection by running the model with the image as input 
# (boxes, scores, classes, num) = sess.run( 
#     [detection_boxes, detection_scores, detection_classes, num_detections], 
#     feed_dict ={image_tensor: image_expanded}) 
  
# # Draw the results of the detection (aka 'visualize the results') 
  
# vis_util.visualize_boxes_and_labels_on_image_array( 
#     image, 
#     np.squeeze(boxes), 
#     np.squeeze(classes).astype(np.int32), 
#     np.squeeze(scores), 
#     category_index, 
#     use_normalized_coordinates = True, 
#     line_thickness = 2, 
#     min_score_thresh = 0.95) 
  
# # All the results have been drawn on the image. Now display the image. 
# cv2.imshow('Object detector', image)
# cv2.imwrite('objectdetection1.jpg', image) 
  
# # Press any key to close the image 
# cv2.waitKey(0) 
  
# # Clean up 
# cv2.destroyAllWindows()


In [1]:
from PIL import Image,ImageFilter

import torchvision.transforms as transforms
from torchvision import *
from torch import *
import cv2 as cv2
import time

from torchvision import *

from torch import *

import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import Dataset, random_split, DataLoader
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.nn as nn
from torchvision.utils import make_grid


In [2]:
# east_path="D:\\Centuriton\\models\\Test\\test1\\Emergency\\6.0.720.mp4"


class ImageClassificationBase(nn.Module):

    def training_step(self, batch):
        images, targets = batch
        out = self(images)
        # _,out = torch.max(out,dim = 1)
        loss = F.binary_cross_entropy(torch.sigmoid(out), targets)
        return loss

   
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()  # Combine losses
        batch_scores = [x['val_score'] for x in outputs]
        epoch_score = torch.stack(batch_scores).mean()  # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_score': epoch_score.item()}

    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_score: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_score']))


class Densenet169(ImageClassificationBase):

    def __init__(self):
        super().__init__()
        self.pretrained_model = models.densenet169(pretrained=True)

        feature_in = self.pretrained_model.classifier.in_features
        self.pretrained_model.classifier = nn.Linear(feature_in, 2)

    def forward(self, x):
        return self.pretrained_model(x)




In [3]:
imsize = (224, 224)
frames_to_skip = 10
li = []
im_output="./"
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                ])

# loading the model
loaded_densenet169 = Densenet169()
loaded_densenet169.load_state_dict(torch.load('./'+'densenet169.pt', map_location=torch.device('cpu')))
loaded_densenet169.eval()

def predict_emergency_vehicle(image_path):
    uploaded_file = image_path

    image = Image.open(uploaded_file).convert('RGB')

    image = image.filter(ImageFilter.MedianFilter)

    image = transform(image).view(1, 3, 224, 224)

    pred = loaded_densenet169.forward(image)
    proba, idx = torch.max(torch.sigmoid(pred), dim=1)

    proba = proba.detach().numpy()[0]
    idx = idx.numpy()[0]

    print("Emergency_Vehicle") if idx == 1 else print("Non_Emergency Vehicle")
    print("Confidence Level",float(proba))
    

# video_reader_east = cv2.VideoCapture(east_path)

tick = time.time()



predict_emergency_vehicle("Test/Test1.jpg")
# predict_emergency_vehicle(im_output+"south.png")
# predict_emergency_vehicle(im_output+"west.png")
# predict_emergency_vehicle(im_output+"north.png")

print('\n\n', 'Time taken: ', time.time() - tick)


C:\Users\manik\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\manik\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet169_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet169_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Non_Emergency Vehicle
Confidence Level 0.9928603768348694


 Time taken:  0.5411326885223389
